# Decision Tree

Trong phần này sẽ tìm hiểu về giải thuật ID3, một giải thuật tham lam cho mô hình Cây quyết định

# Cơ sở lý thuyết

## Các bước trong giải thuật ID3:
* Trường hợp cơ sở:
  * Nếu như tất cả các dữ liệu đang xét thuộc cùng một lớp thì phân lớp cho dữ liệu chính là lớp đó và kết thúc
  * Nếu không còn thuộc tính dùng để phân lớp thì phân lớp cho dữ liệu vào lớp chiếm đa số trong tập dữ liệu đang xét
* Các bước lặp:
  * Lấy một thuộc tính có khả năng phân lớp mạnh nhất cho tập dữ liệu đang xét
  * Phân lớp cho tập dữ liệu theo các giá trị của thuộc tính đó và tạo thành cây con


## Xác định thuộc tính có khả năng phân lớp mạnh nhất:
  * Thuộc tính có khả năng phân lớp mạnh nhất tức có thể chia tập dữ liệu thành các tập con mà trong mỗi tập, số lượng các điểm dữ liệu thuộc mỗi lớp khác nhau là phân biệt đối với nhau
    * Ví dụ: Đối với tập dữ liệu gồm 25 điểm thuộc lớp c1 và 35 điểm thuộc lớp c2, khi phân lớp bằng thuộc tính A sẽ cho ra 3 tập con, trong đó số điểm thuộc lớp c1 và c2 trong mỗi tập lần lượt là (10, 15); (5, 5); (10, 15) thì sẽ được coi là phân loại kém hơn thuộc tính B nếu B chia tập dữ liệu thành 2 tập con (5, 30) và (20, 5)

  * Để đo độ mạnh yếu của thuộc tính phân loại, ta dùng đại lượng "entropy phân loại", entropy phân loại càng cao tức độ hỗn loạn sau khi phân loại càng cao hay nói cách khác các dữ liệu sẽ phân bố gần như là đều với mọi lớp sau khi phân loại thì tính phân loại càng kém:
    * Giả sử thuộc tính A phân loại tập dữ liệu thành các tập con $c_1, c_2, c_3,...,c_n$ với tỉ lệ các phần tử mỗi tập là $q_1, q_2, q_3,...,q_n$ thì entropy phân loại bằng thuộc tính A là: $$H(A) = q_1h(c_1) + q_2h(c_2) + q_3h(c_3) + ... + q_nh(c_n)$$ 
    * Trong đó, với $p_i$ là tỉ lệ các điểm dữ liệu được gán nhãn $i$, entropy thành phần được tính theo công thức: $$h(c_i) = -p_1log(p_1) - p_2log(p_2) - p_3log(p_3) - ... - p_mlog(p_m)$$

# Thực hành code

In [61]:
from typing import List, Dict, TypeVar, Any, NamedTuple, Union
from collections import defaultdict, Counter
import math

## Viết các helper function

### Hàm tính entropy thành phần của một tập
* Đầu vào: Mảng lưu tỉ lệ các nhãn

In [62]:
def entropy(class_probabilities: List[float]) -> float:
    return sum(- p * math.log(p, 2)
               for p in class_probabilities
               if p > 0)

assert entropy([1.0]) == 0
assert entropy([0.5, 0.5]) == 1
assert 0.81 < entropy([0.25, 0.75]) < 0.82

### Hàm tính tỉ lệ các nhãn trong một tập
* Đầu vào: Tập nhãn các điểm dữ liệu
* Counter: Hàm đếm số điểm dữ liệu thuộc một nhãn tương ứng

In [63]:
# Ví dụ về Counter
labels = ['Yes', 'No', 'Yes', 'Yes', 'No']
print("Counter(labels) = ", Counter(labels))
print("Counter(labels).values() = ", Counter(labels).values())
# Hết ví dụ về Counter

Counter(labels) =  Counter({'Yes': 3, 'No': 2})
Counter(labels).values() =  dict_values([3, 2])


In [64]:
def class_probabilities(labels: List[Any]) -> List[float]:
    total_count = len(labels)
    return [count / total_count for count in Counter(labels).values()]

### Hàm tính entropy của một tập nhãn dữ liệu

In [65]:
def data_entropy(labels: List[Any]) -> float:
    return entropy(class_probabilities= class_probabilities(labels= labels))

print("data_entropy(['a']) = ", data_entropy(['a']))
print("data_entropy([True, False]) = ", data_entropy([True, False]))
print("data_entropy(['Yes', 'No', 'Yes', 'Yes', 'No']) = ", data_entropy(['Yes', 'No', 'Yes', 'Yes', 'No']))

data_entropy(['a']) =  0.0
data_entropy([True, False]) =  1.0
data_entropy(['Yes', 'No', 'Yes', 'Yes', 'No']) =  0.9709505944546686


### Hàm tính entropy của một phân loại
* Đầu vào: các tập con sinh ra từ phân loại đó

In [66]:
def partition_entropy(subsets: List[List[Any]]) -> float:
    total_data_point = sum(len(subset) for subset in subsets)
    return sum(len(subset) / total_data_point * data_entropy(subset) for subset in subsets)

### Hàm phân loại dữ liệu theo giá trị thuộc tính

In [67]:
T = TypeVar('T')
def partition_by(data: List[T], attribute: str) -> Dict[Any, List[T]]:
    partition: Dict[Any, List[T]] = defaultdict(list)
    for data_point in data:
        key = getattr(data_point, attribute)
        partition[key].append(data_point)
    return partition

### Hàm tính entropy phân loại theo một thuộc tính

In [68]:
# partitions là một Dict[key = giá trị thuộc tính, value = List[các điểm dữ liệu phân theo giá trị đó]]
# partitions.values() để lấy các điểm dữ liệu đã được phân loại theo giá trị thuộc tính
def partition_entropy_by(data: List[Any],
                         attribute: str,
                         label: str) -> float:
    partitions = partition_by(data= data, attribute= attribute)
    subsets = [[getattr(data_point, label) for data_point in partition] for partition in partitions.values()]
    return partition_entropy(subsets= subsets)

## Biểu diễn dữ liệu

Cần dự đoán kết quả buổi phỏng vấn (did_well) của một ứng viên nếu biết trình độ (level), ngôn ngữ lập trình sử dụng (lang), có dùng tweets không (tweets), có phd không (phd)

In [69]:
from typing import NamedTuple, Optional
class Candidate(NamedTuple):
    level: str
    lang: str
    tweets: str
    phd: bool
    did_well: Optional[bool] = None

In [70]:
                  #  level     lang     tweets  phd  did_well
data = [Candidate('Senior', 'Java',   False, False, False),
          Candidate('Senior', 'Java',   False, True,  False),
          Candidate('Mid',    'Python', False, False, True),
          Candidate('Junior', 'Python', False, False, True),
          Candidate('Junior', 'R',      True,  False, True),
          Candidate('Junior', 'R',      True,  True,  False),
          Candidate('Mid',    'R',      True,  True,  True),
          Candidate('Senior', 'Python', False, False, False),
          Candidate('Senior', 'R',      True,  False, True),
          Candidate('Junior', 'Python', True,  False, True),
          Candidate('Senior', 'Python', True,  True,  True),
          Candidate('Mid',    'Python', False, True,  True),
          Candidate('Mid',    'Java',   True,  False, True),
          Candidate('Junior', 'Python', False, True,  False)
         ]

### Thử phân loại theo các thuộc tính

In [71]:
keys = ['level', 'lang', 'tweets', 'phd']
for key in keys:
    print(f"partition_entropy_by {key}: {partition_entropy_by(data= data, attribute= key, label= 'did_well')}")

partition_entropy_by level: 0.6935361388961918
partition_entropy_by lang: 0.8601317128547441
partition_entropy_by tweets: 0.7884504573082896
partition_entropy_by phd: 0.8921589282623617


## Xây dựng cây quyết định

Cây quyết định được tạo bởi nút Lá (Leaf) chứa giá trị nhãn dự đoán
và các Nhánh (Split) chứa giá trị thuộc tính phân chia (attribute), các cây con được phân chia (subtrees) và giá trị nhãn mặc định được chọn khi xuất hiện giá trị chưa được biết đến (default_value)

Điểm dữ liệu chứa giá trị phân chia chưa được biết đến ở đây có thể là level = "Intern", với default_value = giá trị nhãn chiếm ưu thế hơn, giả sử là True thì điểm dữ liệu này sẽ được gán nhãn là True 

Ở đây cây con là một dict có keys là các giá trị của thuộc tính phân chia (attribute), values là những điểm mà thuộc tính phân chia có giá trị đó

In [72]:
class Leaf(NamedTuple):
    value: Any

class Split(NamedTuple):
    attribute: str
    subtrees: dict
    default_value: Any = None

DecisionTree = Union[Leaf, Split]
    

### Hàm phân loại cho cây quyết định

In [73]:
def classify(tree: DecisionTree, input: Any) -> Any:
    # Nếu là một nút lá thì trả về giá trị dự đoán
    if isinstance(tree, Leaf):
        return tree.value
    
    # Nếu không
    # Lấy giá trị của thuộc tính phân chia trong input
    subtree_key = getattr(input, tree.attribute)
    # Nếu giá trị thuộc tính này chưa từng có trong tập huấn luyện
    # thì trả về nhãn ưu thế nhất
    if subtree_key not in tree.subtrees:
        return tree.default_value
    # Nếu đã có trong tập huấn luyện thì phân chia tiếp theo giá trị này
    subtree = tree.subtrees[subtree_key]
    return classify(tree= subtree, input= input)

### Hàm xây dựng cây quyết định

Đầu vào:
* Tập dữ liệu huấn luyện (data)
* Thuộc tính phân chia (split_attribute)
* Thuộc tính cần dự đoán (target_attribute)

In [ ]:
def build_tree_id3(data: List[Any],
                   split_attribute: List[str],
                   target_attribute: str) -> DecisionTree:
    
    labels_count = Counter(getattr(data_point, target_attribute) for data_point in data)
    most_common_label = labels_count.most_common(1)[0][0]

    if len(labels_count) == 1:
        return Leaf(most_common_label)
    if not split_attribute:
        return Leaf(most_common_label)
    
    def split_entropy(attibute: str) -> float:
        return partition_entropy_by(data= data, attribute= attibute, label= target_attribute)
    
    best_attribute = min(split_attribute, key= split_entropy)

    partitions = partition_by(data= data, attribute= best_attribute)

    new_attribute = [a for a in split_attribute if a != best_attribute]

    subtrees = {attribute_value: build_tree_id3(data= subset,
                                               split_attribute= new_attribute,
                                               target_attribute= target_attribute)
                for attribute_value, subset in partitions.items()}
    
    return Split(attribute= best_attribute, 
                 subtrees= subtrees, 
                 default_value= most_common_label)

In [75]:
tree = build_tree_id3(data= data, 
                      split_attribute= ['level', 'lang', 'tweets', 'phd'],
                      target_attribute= 'did_well')

In [76]:
classify(tree= tree, input= Candidate("Junior", "Java", True, False))

True

In [77]:
classify(tree= tree, input= Candidate("Junior", "Java", True, True))

False

In [78]:
classify(tree= tree, input= Candidate("Intern", "Java", True, True))

True